<a href="https://colab.research.google.com/github/1kaiser/test2023/blob/main/TestingVisualizingWeights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pathlib
dir='/content/files'
urls = 'https://github.com/1kaiser/Snow-cover-area-estimation/releases/download/v1/imagesfolder.zip'
data_dir = tf.keras.utils.get_file(origin=urls,
                                   fname='s',
                                   cache_subdir= dir,
                                   archive_format='auto',
                                   untar=False,
                                   extract=True)
!rm -r {dir}/s
data_dir = pathlib.Path(data_dir)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os.path
import re

from osgeo import gdal

def tif2array(input_file, calc_gain=True):

    dataset = gdal.Open(input_file, gdal.GA_ReadOnly)
    image_datatype = dataset.GetRasterBand(1).DataType
    image = np.zeros((dataset.RasterYSize, dataset.RasterXSize, dataset.RasterCount), dtype=float)

    return image, dataset


In [ ]:
image_dir = r'/content/files/'

#############################################################################
prefix = "sur_refl_"
end = ["b01", "b02", "b03", "b04", "b05", "b06", "b07", "day_of_year", "qc_500m", "raz", "state_500m", "szen", "vzen"]
DayOY = "_doy\[0-9]+_aid0001"
fileExt = r'.tif'
expression_b1 = prefix+end[0]
expression_b2 = prefix+end[1]
expression_b3 = prefix+end[2]
expression_b4 = prefix+end[3]
expression_b5 = prefix+end[4]
expression_b6 = prefix+end[5]
expression_b7 = prefix+end[6]


imgs_list_b1 = [f for f in os.listdir(image_dir) if f.__contains__(expression_b1)]

imgs_list_b1.sort(reverse=True)       

In [ ]:
from google.colab import output
temp_dir = r'/content/'

def ybatchedimages(images_path, image_list, batch_idx):
  images = []
  path = os.path.join(images_path, image_list[batch_idx])
  pathb2 = path.replace(expression_b1, expression_b2)
  pathb4 = path.replace(expression_b1, expression_b4)
  pathb6 = path.replace(expression_b1, expression_b6)

  #creating file NDSI
  !gdal_calc.py \
    --overwrite \
    --type=Float32 \
    --NoDataValue=0 \
    -A {pathb4} \
    --A_band 1 \
    -B {pathb6} \
    --B_band 1 \
    -C {pathb2} \
    --C_band 1 \
    --outfile={temp_dir}"BothCheck_result_final.tif" \
    --calc="(((A.astype(float) - B)/(A.astype(float) + B))>=0.4)*(C.astype(float)/10000>0.11)"

  pathout = temp_dir+str('BothCheck_result_final.tif')
  images.append(tif2array(pathout, 0)[0])

  !rm -r {temp_dir}"BothCheck_result_final.tif"
  output.clear()
  return images


import jax.numpy as jnp
def xbatchedimages(images_path, image_list, batch_idx):
  images = []
  path = os.path.join(images_path, image_list[batch_idx])
  v1 = tif2array(path.replace(expression_b1, expression_b1),0)[0]
  v2 = jnp.append(v1,tif2array(path.replace(expression_b1, expression_b2),0)[0] , axis =2)
  v3 = jnp.append(v2, tif2array(path.replace(expression_b1, expression_b3),0)[0] , axis =2)
  v4 = jnp.append(v3, tif2array(path.replace(expression_b1, expression_b4),0)[0] , axis =2)
  v5 = jnp.append(v4, tif2array(path.replace(expression_b1, expression_b5),0)[0] , axis =2)
  v6 = jnp.append(v5, tif2array(path.replace(expression_b1, expression_b6),0)[0] , axis =2)
  v7 = jnp.append(v6, tif2array(path.replace(expression_b1, expression_b7),0)[0] , axis =2)
  images.append(v7)
  w1 = tif2array(path.replace(expression_b1, expression_b1),0)[0]
  w2 = tif2array(path.replace(expression_b1, expression_b2),0)[0]
  w3 = tif2array(path.replace(expression_b1, expression_b3),0)[0]
  w4 = tif2array(path.replace(expression_b1, expression_b4),0)[0]
  w5 = tif2array(path.replace(expression_b1, expression_b5),0)[0]
  w6 = tif2array(path.replace(expression_b1, expression_b6),0)[0]
  w7 = tif2array(path.replace(expression_b1, expression_b7),0)[0]
  return images

import jax.random as random
import jax.numpy as jnp
batch_size = 1
no_of_batches = int(len(imgs_list_b1)/batch_size)
def data_stream(i, no_of_batches):
  return jnp.asarray(xbatchedimages(image_dir, imgs_list_b1, i)), jnp.asarray(ybatchedimages(image_dir, imgs_list_b1, i))

In [ ]:
#✅
import jax
import jax.numpy as jnp


positional_encoding_dims = 6  # Number of positional encodings applied

def positional_encoding(args):
    image_height_x_image_width, cha = args.shape
    inputs_freq = jax.vmap(lambda x: args * 2.0 ** x)(jnp.arange(positional_encoding_dims))
    x = jnp.stack([jnp.sin(inputs_freq), jnp.cos(inputs_freq)])
    x = x.swapaxes(0, 2)
    x = x.reshape([image_height_x_image_width, -1])
    x = jnp.concatenate([args, x], axis=-1)
    return x

def batch_encoded(args):
    img_list = []
    for i in range(args.shape[0]):
        c = args[i]
        c = c.reshape(-1, c.shape[2])
        p = positional_encoding(c)
        img_list.append(p.reshape(args.shape[1],args.shape[2],p.shape[1]))
        x = jnp.array(img_list)
    return x

#✅
!python -m pip install -qq -U flax orbax
# Orbax needs to enable asyncio in a Colab environment.
!python -m pip install -qq nest_asyncio


import jax
import jax.numpy as jnp

import flax
import optax
from typing import Any

from jax import lax
import flax.linen as nn
from flax.training import train_state, common_utils

apply_positional_encoding = True # Apply posittional encoding to the input or not
ndl = 8 # num_dense_layers Number of dense layers in MLP
dlw = 256 # dense_layer_width Dimentionality of dense layers' output space 

##########################################<< MLP MODEL >>#########################################
class MLPModel(nn.Module):
    dtype: Any = jnp.float32
    precision: Any = lax.Precision.DEFAULT
    apply_positional_encoding: bool = apply_positional_encoding
    @nn.compact
    def __call__(self, input_points):
      x = batch_encoded(input_points) if self.apply_positional_encoding else input_points
      for i in range(ndl):
          x = nn.Dense(dlw,dtype=self.dtype,precision=self.precision)(x)
          x = nn.relu(x)
          x = jnp.concatenate([x, input_points], axis=-1) if i == 4 else x
      x = nn.Dense(1, dtype=self.dtype, precision=self.precision)(x)
      return x
##########################################<< MLP MODEL >>#########################################

#✅
!python -m pip install -q -U flax
import optax
from flax.training import train_state
import jax.numpy as jnp
import jax


def Create_train_state(r_key, model, shape, learning_rate ) -> train_state.TrainState:
    print(shape)
    variables = model.init(r_key, jnp.ones(shape)) 
    optimizer = optax.adam(learning_rate) 
    return train_state.TrainState.create(
        apply_fn = model.apply,
        tx=optimizer,
        params=variables['params']
    )

learning_rate = 1e-4
model = MLPModel() # Instantiate the Model

In [ ]:
batches = data_stream(1, no_of_batches)
BATCH, H, W, Channels = batches[0].shape
state = Create_train_state( rng, model, (BATCH, H, W, Channels ), learning_rate ) 
count = 1